# 1) Solving the Lorenz System and plotting it as nice as you can

\begin{aligned}
{\frac {\mathrm {d} x}{\mathrm {d} t}}&=\sigma (y-x),\\[6pt]
{\frac {\mathrm {d} y}{\mathrm {d} t}}&=x(\rho -z)-y,\\[6pt]
{\frac {\mathrm {d} z}{\mathrm {d} t}}&=xy-b z.
\end{aligned}
Whith values  $\sigma=10, \rho=28, b=2.667$

* Implement the Lorenz system of differential equations
* Solve the Lorenz system of PDEs using scipy
* Make three subplots with x, y, z as a function of the time
* try to make the plots as complete as you can
* Extra:
    - maybe try to plot in 3d the trajectory of the system
    - implement yourself an ode-integrator and check the runtime difference compared to scipy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

def lorenz(x, t, sigma=10, rho=28, b=2.667):
    x_dot = sigma*(x[1] - x[0])
    y_dot = rho * x[0] - x[1] - x[0] * x[2]
    z_dot = x[0] * x[1] - b * x[2]
    return x_dot, y_dot, z_dot

time = np.arange(0, 100, 0.01)
trajectory = odeint(lorenz, y0=[0, 1, 1], t=time)

# params
slice_to_show = slice(100, 5000)
y_axis_names = ['x(t)', 'y(t)', 'z(t)']

fig, axes = plt.subplots(3, 1, figsize=(10, 10))

fig.suptitle('Lorenz dynamics')
for i, trajectory_x in enumerate(trajectory.T):
    axes[i].plot(time[slice_to_show],
                 trajectory_x[slice_to_show],
                 c=np.random.rand(3)
                )
    axes[i].set_ylabel(y_axis_names[i])

ax = plt.figure(figsize=(10, 10)).add_subplot(projection='3d')

ax.plot(trajectory[slice_to_show, 0],
        trajectory[slice_to_show, 1],
        trajectory[slice_to_show, 2])
ax.set_title("Lorenz Attractor")
plt.show()


In [ ]:
def euler_step(func, y, dt=1e-3):
    df_dx = func(y, 0)
    y = y + np.array(df_dx) * dt
    return y

def integrate_euler(func, y0, dt=5e-3, time_steps=100000):
    euler_trajectory = np.zeros((time_steps, len(y0)))
    euler_trajectory[0] = y0
    for i in range(time_steps - 1):
        euler_trajectory[i + 1] = euler_step(func, euler_trajectory[i], dt=dt) 
    return euler_trajectory
                        
%time trajectory = odeint(lorenz, y0=[1, 0, 0], t=time)
%time new_trajectory = integrate_euler(lorenz, y0=[1, 0, 0], time_steps=len(time))

ax = plt.figure(figsize=(10, 10)).add_subplot(projection='3d')

ax.plot(trajectory[:, 0],
        trajectory[:, 1],
        trajectory[:, 2]
       )

ax.plot(new_trajectory[:, 0],
        new_trajectory[:, 1],
        new_trajectory[:, 2]
       )
ax.set_title("Lorenz Attractor")
plt.show()

# 2) Image restoration

* Load the `image_gray` file in data using matplotlib `np.load`
* Visualize the image, you will see that the image as some `burned` pixels where the value is exactly $1$
* Find all `burned` pixels
* Replace those values with the average of neighbors pixel
* Extras:
    - If you used for loops try to find a way to do the same without, and check if it's faster.
    - Adapt the same restoration for the color image `image_rgb`
    

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

image = np.load('./data/image_gray.npy')
plt.figure(figsize=(10, 10))
plt.imshow(image, cmap='gray')
plt.show()

In [ ]:
from scipy.ndimage import convolve

smoothing_filter = np.ones((3, 3))
smoothing_filter /= np.sum(smoothing_filter)

smooth_image = convolve(image, smoothing_filter)

plt.figure(figsize=(10, 10))
plt.imshow(smooth_image, cmap='gray')
plt.show()

In [ ]:
# method 1
mask = image == 1
plt.imshow(mask, interpolation='nearest')

In [ ]:
# method 1
mask = image == 1
image[mask] = smooth_image[mask]

In [ ]:
# method 2
new_image = np.where(image == 1, smooth_image, image)

In [ ]:
# method 3 (do not use this)
for i in range(image.shape[0]):
    for j in range(image.shape[1]):
        pixel = image[i, j]
        if pixel == 1:
            image[i, j] = smooth_image[i, j]

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(new_image, cmap='gray')
plt.show()

In [ ]:
from scipy.ndimage import convolve

smoothing_filter = np.ones((5, 5)) / (5*5)
smooth_image = convolve(image, smoothing_filter)
mask = image == 1
image[mask] = smooth_image[mask]

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(image, cmap='gray')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
image = np.load('./data/image_rgb.npy')
plt.figure(figsize=(10, 10))
plt.imshow(image)
plt.show()

In [ ]:
def l2_distance(x, y):
    dist = (x - y[None, None, :])**2
    dist = np.sum(dist, axis=2)
    dist = np.sqrt(dist)
    return dist
    
image = np.load('./data/image_rgb.npy')
smoothing_filter = np.ones((5, 5, 1))
smoothing_filter /= np.sum(smoothing_filter)

smooth_image = convolve(image, smoothing_filter)

for color in [[1, 0, 0], [0, 1, 0], [0, 0, 1]]:
    color = np.array(color)
    # mask = image == color[None, None, :]
    # Much safer!!! 
    mask = l2_distance(image, color) < 0.00001
    image[mask] = smooth_image[mask]

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(image)

In [ ]:
image = np.load('./data/image_rgb.npy')

smoothing_filter = np.ones((5, 5, 1)) / (5*5)
smooth_image = convolve(image, smoothing_filter)

colors = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

mask = image[:, :, None, :] == colors[None, None, ...]
mask = np.any(mask, axis=3)
image[mask] = smooth_image[mask]

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(image)